<a href="https://colab.research.google.com/github/mansiag/LearnDL/blob/master/char_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import TimeDistributed, Dense, Activation
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from keras.layers import Dropout
import os

Using TensorFlow backend.


In [0]:
#Upload the dataset
from google.colab import files
uploaded = files.upload()

Saving HP.txt to HP.txt


**Data** **Preprocessing**

In [0]:
data = open("HP.txt","r").read()
vocab = list(set(data))
data_size, vocab_size = len(data), len(vocab)

In [0]:
print(data_size, vocab_size)
print(vocab)

437326 79
['B', '"', 'D', 'e', 'M', ',', 'm', 'j', '!', ')', '6', '0', '\ufeff', 'F', 'w', 'W', '4', 'i', 'S', "'", 'L', '\\', '~', '3', 'X', 'T', 'A', 'f', '*', 'C', 'R', 's', ' ', 'o', '7', 'V', 'd', 'Y', '1', 'H', '8', 'U', '(', 'P', 'h', 'N', 'E', 'z', 'O', 'Q', 'I', '-', 'b', 'p', 'g', '9', 'K', 'x', '.', ';', 'q', '\n', ':', 'r', 'y', '?', '2', '5', 'a', 't', 'l', 'k', 'J', 'n', 'Z', 'u', 'G', 'c', 'v']


In [0]:
char_to_ix = {ch:i for i,ch in enumerate(vocab)}
ix_to_char = {i:ch for i,ch in enumerate(vocab)}
timestep = 40

In [0]:
#Vectorization
count = data_size - timestep
inp = []
output = []
for i in range(count):
  inp.append(data[i:i+timestep])
  output.append(data[i+1:i+timestep+1])
  
X = np.zeros((len(inp),timestep,vocab_size), dtype=bool)
y = np.zeros((len(output),timestep,vocab_size), dtype=bool)
for i,sentence in enumerate(inp):
  for j,k in enumerate(sentence):
    X[i, j, char_to_ix[k]] = 1             #One-hot Encoding
    y[i, j, char_to_ix[output[i][j]]] = 1

In [0]:
#Creating the model
def get_model():
  model = Sequential()
  model.add(LSTM(256,input_shape = (timestep, vocab_size),activation='tanh', return_sequences=True)) #here we have used return_sequences=True to get the full sequences of successive outputs for each timestep
  model.add(Dropout(0.2))
  model.add(TimeDistributed(Dense(vocab_size)))  #remember to use TimeDistributed to get the full sequences of successive outputs for each timestep
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  return model

In [0]:
#Training
model = get_model()
from keras.models import Sequential
model.fit(X,y, epochs=50, batch_size=200)


Epoch 1/50
437286/437286 [==============================] - 242s 553us/step - loss: 2.1197
Epoch 2/50
437286/437286 [==============================] - 241s 551us/step - loss: 1.5985
Epoch 3/50
437286/437286 [==============================] - 240s 549us/step - loss: 1.4404
Epoch 4/50
437286/437286 [==============================] - 241s 551us/step - loss: 1.3515
Epoch 5/50
437286/437286 [==============================] - 241s 551us/step - loss: 1.2933
Epoch 6/50
437286/437286 [==============================] - 239s 547us/step - loss: 1.2522
Epoch 7/50
437286/437286 [==============================] - 240s 549us/step - loss: 1.2220
Epoch 8/50
437286/437286 [==============================] - 240s 549us/step - loss: 1.1991
Epoch 9/50
437286/437286 [==============================] - 240s 548us/step - loss: 1.1811
Epoch 10/50
437286/437286 [==============================] - 237s 542us/step - loss: 1.1663
Epoch 11/50
437286/437286 [==============================] - 238s 545us/step - loss: 1.15

Above, the model gets trained only for 28 epochs because the space of the system is full because we are doing character level one hot encoding

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 40, 256)           344064    
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 79)            20303     
_________________________________________________________________
activation_2 (Activation)    (None, 40, 79)            0         
Total params: 364,367
Trainable params: 364,367
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Prediction
#Creating input vector for prediction
sample_data = input('Enter Text')


Enter Texthagrid is helpful to harry potter. he is hagrid. hagrid


In [0]:
def one_hot_encoding(data):
  test_X = np.zeros((1,timestep,vocab_size), dtype=bool)
  for j, alphabet in enumerate(data):
    test_X[0,j,char_to_ix[alphabet]] = 1
  return test_X

In [0]:
if len(sample_data) > timestep:
  input_sentence = sample_data[len(sample_data)-timestep:-1]
  test_X = one_hot_encoding(input_sentence)
else:
  test_X = one_hot_encoding(sample_data)


In [0]:
to_predict = 400
generated = list(sample_data)

for i in range(to_predict):
  preds = model.predict(test_X)
  next_char_index = np.argmax(preds[0][-1])
  next_char = ix_to_char[next_char_index]
  generated.append(next_char)
  test_X = one_hot_encoding(generated[len(generated)-timestep:-1])
  
print(str(generated))

['h', 'a', 'g', 'r', 'i', 'd', ' ', 'i', 's', ' ', 'h', 'e', 'l', 'p', 'f', 'u', 'l', ' ', 't', 'o', ' ', 'h', 'a', 'r', 'r', 'y', ' ', 'p', 'o', 't', 't', 'e', 'r', '.', ' ', 'h', 'e', ' ', 'i', 's', ' ', 'h', 'a', 'g', 'r', 'i', 'd', '.', ' ', 'h', 'a', 'g', 'r', 'i', 'd', 'k', ' ', 'e', 'h', 't', 'a', ' ', 'd', 'a', 'e', 'k', '.', 'i', '.', ' ', ' ', 'o', 'e', ' ', 'n', 'o', 'i', ',', 'e', ' ', 'h', 'o', ' ', 's', 'a', 'o', 'd', ' ', ' ', 'h', 'u', ' ', 'd', 'a', 'e', ' ', '.', 'o', '.', 't', ' ', 'n', 'e', 'a', 't', ' ', 'e', 'i', 't', 'h', 'e', 'r', ' ', ' ', 'i', 'e', ' ', 't', 'o', 'e', 'e', ' ', 't', 'o', 'a', 'a', 't', ',', ' ', ' ', 'o', 'e', ' ', 'n', 'o', 'e', 'm', 'c', 's', 'l', 'l', 'y', '-', 's', 'i', 'u', ' ', ' ', 'e', 'e', 't', ' ', ' ', 'o', 'r', 'r', 's', 'b', 'l', 'a', 'g', 'e', 'e', 'h', 'T', 'a', 'd', 's', 's', "'", 's', ' ', 's', 'r', 'i', 'e', 'c', ' ', ' ', 'o', 'a', ' ', 's', 'a', 'a', 'd', ',', '-', "'", 'a', 'm', ' ', 'i', 'o', ' ', 's', 'a', 'i', 'd', ' ',

In [0]:
''.join(generated)

'hagrid is helpful to harry potter. he is hagrid. hagridk ehta daek.i.  oe noi,e ho saod  hu dae .o.t neat either  ie toee toaat,  oe noemcslly-siu  eet  orrsblageehTadss\'s sriec  oa saad,-\'am io said  oo  oer.a.  oh ae poot,a hooslls.s"ese .o.m\'o  eet air aadp taok ao eess lutg io sard,-"a-  refsea ,  ae  aeg eog erge. "either  oart  oe coar reat niwe e-v rtan  oa toieg .a toidh. \'oy  ao  eet  oe  oe .o.myon  oehsaods\'ov rrlettoee  raasloy,  aet aig e'

"['o', ' ', 'm', 'y', ' ']"